In [2]:
import hyp3_sdk as sdk
from hyp3_sdk import HyP3
import getpass
from datetime import datetime
import os
import sys
sys.path.insert(1, '../')
from utils import *

In [ ]:
uid = getpass.getpass("Enter your EathData user id?")
pswd = getpass.getpass("Enter your EathData password?")
hyp3 = HyP3(username=uid, password=pswd)

In [ ]:
project_name = f"AN-AU-coreg-test-20m-{datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}"

with open(f"../data/asf/sub1/bbox.txt", "r") as f:
    bbox = f.readline().strip()
    s2_id = f.readline().strip()

s1_scene_files = get_s1_filenames([bbox], ["2023", "2024"], f"../data/asf/sub1/scenes.txt", ["SLC"])
# get the Dec 2024 for S1 GRD
scene = s1_scene_files[0]

download_folder = f"../data/asf/{s2_id}/sub1/"

rtc_job = sdk.Batch()
rtc_job = hyp3.submit_rtc_job(
    os.path.splitext(os.path.basename(scene))[0], 
    include_dem=True, #include dem in final product
    include_inc_map=True,  #include dem map in final product
    include_rgb=True,  #include rgb img in final product
    include_scattering_area=True, #include scat area in final product
    name=project_name,
    resolution=20,
    dem_name='copernicus',
    radiometry='gamma0',
)
print(rtc_job)

# watch job
rtc_job = hyp3.find_jobs(name=project_name)
rtc_job = hyp3.watch(rtc_job)

# download when done
rtc_file = rtc_job.download_files(download_folder)

with open(f"../data/inputs/{s2_id}/sub1/rtc.txt", "w") as f:
    f.write(f"{str(rtc_file[0])}")